# Qiskit: گیت‌های چندکیوبیتی

In [ ]:
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
from qiskit.quantum_info import random_statevector
import numpy as np
import warnings
warnings.filterwarnings('ignore')

sim = Aer.get_backend('aer_simulator')
%matplotlib

In [ ]:
def get_statevector(qc: QuantumCircuit) -> np.ndarray:
    sim = Aer.get_backend('aer_simulator')
    qc.save_statevector()
    qobj = assemble(qc)
    state = sim.run(qobj).result().get_statevector(qc)
    return state

منطقا کیوبیت‌هایی که با هم کاری نداشته‌باشن نمی‌تونن خیلی محاسبات عجیبی انجام بدن، برای همینم می‌خوایم یه کاری کنیم کیوبیت‌هامون برهم‌کنش داشته‌باشن و سعی کنیم محاسباتی فراتر از محاسبات کلاسیک انجام بدیم.

## ۱. یادآوری: نشون‌دادن وضعیت‌های چندکیوبیتی

یادمون هست که وضعیت یه سیستم دو کیوبیتی رو می‌تونستیم با استفاده از ضرب تانسوری و به صورت زیر تعریف کنیم:

$$ |a\rangle = a_{00}|00\rangle + a_{01}|01\rangle + a_{10}|10\rangle + a_{11}|11\rangle = \begin{bmatrix} a_{00} \\ a_{01} \\ a_{10} \\ a_{11} \end{bmatrix} $$

که این‌جا $ a_{ij} $ ها اعداد مختلطی تحت این شرایط بودن:

$$ |a_{00}|^2 + |a_{01}|^2 + |a_{10}|^2 + |a_{11}|^2 = 1$$

و قوانین اندازه‌گیری هم مثل قبل بهشون اعمال می‌شد:
$$ p(|ij\rangle) = |\langle ij | a \rangle |^2 = |a_{ij}|^2 \space ; \space i, j \in \{0, 1\}$$

اگه ما دوتا کیوبیت جدا داشته‌باشیم، می‌تونیم بردار وضعیت‌شونو به این شکل توصیف کنیم:

$$ |a\rangle = \begin{bmatrix} a_0 \\ a_1 \end{bmatrix}, \quad |b\rangle = \begin{bmatrix} b_0 \\ b_1 \end{bmatrix} $$

$$ 
|ba\rangle = |b\rangle \otimes |a\rangle = \begin{bmatrix} b_0 \times \begin{bmatrix} a_0 \\ a_1 \end{bmatrix} \\ b_1 \times \begin{bmatrix} a_0 \\ a_1 \end{bmatrix} \end{bmatrix} = \begin{bmatrix} b_0 a_0 \\ b_0 a_1 \\ b_1 a_0 \\ b_1 a_1 \end{bmatrix}
$$

و اگه سه تا کیوبیت داشته‌باشیم:
$$ 
|cba\rangle = \begin{bmatrix} c_0 b_0 a_0 \\ c_0 b_0 a_1 \\ c_0 b_1 a_0 \\ c_0 b_1 a_1 \\
                              c_1 b_0 a_0 \\ c_1 b_0 a_1 \\ c_1 b_1 a_0 \\ c_1 b_1 a_1 \\
              \end{bmatrix}
$$

پس همون‌طور که می‌بینید اگه $n$تا کیوبیت داشته‌باشیم، برای دونستن وضعیتش باید $2^n$تا عدد مختلط رو نگه داریم. چون این اندازه به صورت نمایی افزایش پیدا می‌کنه، شبیه‌سازی رفتار کامپیوترهای کوانتومی روی کامپیوترهای کلاسیک سختن. مثلا یه لپ‌تاپ امروزی می‌تونه یه بردار وضعیت با حدود ۲۰ کیوبیت رو به راحتی شبیه‌سازی کنه، ولی شبیه‌سازی ۱۰۰  کیوبیت برای بزرگ‌ترین ابرکامیپوترها هم بیش از حد سخته.

برای یه دست‌گرمی ساده، سعی کنید وضعیت زیر رو به صورت ضرب تانسوری دوتا کیوبیت بنویسید.

$$|\psi\rangle = \frac{1}{\sqrt{2}}|00\rangle + \frac{i}{\sqrt{2}}|01\rangle $$

به عنوان یه مثال ساده، مدار زیر رو ببینید:

In [ ]:
qc = QuantumCircuit(3)
# Apply H-gate to each qubit:
for qubit in range(3):
    qc.h(qubit)
# See the circuit:
qc.draw(output="mpl")

که وضعیت همه‌ی کیوبیت‌ها رو به استیت  $|+\rangle$ می‌بره، یعنی وضعیت نهایی‌مون به صورت زیر درمی‌آد:

$$ 
|{+++}\rangle = \frac{1}{\sqrt{8}}\begin{bmatrix} 1 \\ 1 \\ 1 \\ 1 \\
                              1 \\ 1 \\ 1 \\ 1 \\
              \end{bmatrix}
$$

و نتیجه‌ش رو هم می‌تونیم با اجرای این کد تایید کنیم:

In [ ]:
# Let's see the result
state = get_statevector(qc)
state_vertical = state.reshape(8, 1)
array_to_latex(state_vertical, prefix="\\text{Statevector} = ")

## ۲. گیت‌های چندکیوبیتی

### ۱.۲ گیت‌های چندکیوبیتی ترکیبی

همون‌طور که یه سیستم $n$ کیوبیتی رو با یه بردار $2^n$ کیوبیتی نشون می‌دیم، برای اعمال تغییرات روشون به ماتریس‌هایی  با ابعاد $2^n \times 2^n $ نیاز داریم.
بعضی از این ماتریس‌ها رو می‌شه به صورت ضرب تانسوری دوتا گیت تک‌کیوبیتی نوشت. به عنوان مثال داریم:

$$
X|q_1\rangle \otimes H|q_0\rangle = (X\otimes H)|q_1 q_0\rangle
$$

که ماتریسش به این صورته

$$
X\otimes H = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \otimes \tfrac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} = \frac{1}{\sqrt{2}}
\begin{bmatrix} 0 \times \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
              & 1 \times \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
                \\ 
                1 \times \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
              & 0 \times \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
\end{bmatrix} = \frac{1}{\sqrt{2}}
\begin{bmatrix} 0 & 0 & 1 & 1 \\
                0 & 0 & 1 & -1 \\
                1 & 1 & 0 & 0 \\
                1 & -1 & 0 & 0 \\
\end{bmatrix}
$$

این گیت‌ها رو می‌شه عملا به این صورت دید که داریم دوتا گیت تک‌کیوبیتی رو همزمان روی دوتا کیوبیت اجرا می‌کنیم.
توی Qiskit، ما می‌تونیم مداری که درست می‌کنیم رو به صورت یه ماتریس یکانی نشون بدیم. مثلا برای همین گیت $ (X\otimes H) $ داریم: 

In [ ]:
qc = QuantumCircuit(2)
qc.h(0)
qc.x(1)
qc.draw(output="mpl")

In [ ]:
qc.save_unitary()
qobj = assemble(qc)
unitary = sim.run(qobj).result().get_unitary()

In [ ]:
array_to_latex(unitary, prefix="\\text{Circuit = }\n")

### ۲.۲. گیت CNOT

گیت CNOT یه گیت چندکیوبیتیه و گیت نسبتا خاصی‌ه، چراکه کاری که می‌کنه کاملا به این بستگی داره که وضعیت یکی از کیوبیت‌هامون در لحظه‌ی اجرا چی باشه، و به همین دلیل نمی‌شه به صورت ضرب تانسوری دوتا گیت تک‌کیوبیتی یکانی نوشتنش.

این گیت، یه گیت $X$ یا $NOT$ رو روی یکی از کیوبیت‌ها (که بهش می‌گیم ‍`target`) اعمال می‌کنه، تنها در صورتی که اون‌یکی کیوبیت (که بهش می‌گیم `control`) توی حالت $|1\rangle$ باشه. 

نمایش ماتریسی $CNOT$ ای که توش کیوبیت اول `control` ه و کیوبیت دومی `target` هم به این شکله:

$$
\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\
                              0 & 1 & 0 & 0 \\
                              0 & 0 & 0 & 1 \\
                              0 & 0 & 1 & 0 \\
              \end{bmatrix}
$$

و اگه کیوبیت اول `target` باشه و کیوبیت دومی `control`، $CNOT$ به این صورت در می‌آد.

$$
\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\
                              0 & 0 & 0 & 1 \\
                              0 & 0 & 1 & 0 \\
                              0 & 1 & 0 & 0 \\
              \end{bmatrix}
$$

به این معنا که $q_0$ کیوبیت `control` مون‌ه و $q_1$ کیوبیت `target`مون.

یه نحوه‌ی دیگه فکر کردن بهش اینه که $CNOT$ وقتی روی یه بردار ۴-بعدی (۲ کیوبیتی) اجرا بشه، جای ضریب‌های این دوتا وضعیت پایه رو با هم عوض می‌کنه:

$$ 
|a\rangle = \begin{bmatrix} a_{00} \\ a_{01} \\ a_{10} \\ a_{11} \end{bmatrix}, \quad \text{CNOT}|a\rangle = \begin{bmatrix} a_{00} \\ a_{01} \\ a_{11} \\ a_{10} \end{bmatrix} \begin{matrix} \\ \\ \leftarrow \\ \leftarrow \end{matrix}
$$

این گیت توی Qiskit به صورت `cx(control, target)` پیاده‌سازی شده.

### تمرین 💡

۱. می‌دونیم که گیت $CNOT$ روی دوتا کیوبیت control و target عمل می‌کنه. چک کنید و ببینید اگه کیوبیت controlمون توی superposition نباشه، می‌تونیم با CNOT کاری انجام بدیم که قبلا هم می‌تونستیم انجام بدیم یا نه.
و اون حالت رو با حالتی که کیوبیت `control` توی superpositionای از $ |0\rangle$ و $|1\rangle$ باشه مقایسه کنید. 


۲. توی جلسه‌ی دست به کار روز اول، با چهار بردار متعامد یکه آشنا شدیم که یه پایه‌ی برای فضای برداری‌مون تشکیل 
می‌دادن. به این بردارها که این زیر می‌بینیم‌شون Bell State می‌گن.

$$|{\Phi_\pm}\rangle = \frac{1}{\sqrt 2}\big(|0\rangle| 0\rangle\pm |1\rangle| 1\rangle\big), \qquad |{\Psi_{\pm}}\rangle=\frac{1}{\sqrt 2}\big(|0\rangle| 1\rangle\pm  |1\rangle|0\rangle\big).$$

با کمک گیت‌هایی که تا الان بهتون معرفی شده، یکی از Bell Stateها رو بسازید و ببینید با استفاده از گیت‌های تک‌کیوبیتی چه‌طور می‌تونید بردارهای این پایه رو به هم تبدیل کنید.



۳. یه گیت معروف دیگه، گیت $SWAP$ هست که همون‌طور که از اسمش مشخصه، دوتا کیوبیت رو می‌گیره و اون‌ها رو با هم جابه‌جا می‌کنه، یعنی بردار کیوبیت ۱ رو تبدیل به بردار کیوبیت ۲ می‌کنه و بالعکس. نمایش ماتریسی این گیت رو این زیر می‌بینید:

$$
\text{SWAP} = \begin{bmatrix} 
                                1 & 0 & 0 & 0 \\
                              0 & 0 & 1 & 0 \\
                              0 & 1 & 0 & 0 \\
                              0 & 0 & 0 & 1 \\
              \end{bmatrix}
$$
سعی کنید این گیت رو با استفاده از گیت‌هایی که تا حالا بهتون معرفی شده بسازید.

۴. بعضی کامپیوترهای کوانتومی به خاطر محدودیت‌های سخت‌افزاری که دارن، فقط می‌تونن CNOT رو در یه جهت اعمال کنن، یعنی فقط می‌تونن `cx(0, 1)` رو انجام بدن. آیا می‌شه کاری کرد که بدون استفاده‌ی مستقیم از `cx(1, 0)‍‍` این کارکرد رو پیاده‌سازی کرد؟ اگه آره، چطور؟ اگه نه، چرا؟

## تمرین‌های سخت!  🧠

حالا، یه سری بردار وجود دارن که درست کردنشون به همین سادگی‌ها هم نیست، ولی می‌شه از بردار $|0\rangle^{\otimes n} $ شروع کرد و با استفاده از گیت‌هایی که تا الان شناختیم، ساخت‌شون.

خیلی نگران نباشید، با هم‌فکری و یه مقدار بازی‌کردن با جبرخطی‌ش، می‌تونید راحت استیت‌ها رو بسازید.

برای همه‌ی سوال‌های این قسمت، یه تابع بنویسید که ورودی‌ای که تو سوال اومده رو بگیره و خروجی‌ای که اومده رو ریترن کنه. (خروجی می‌تونه به صورت یه آبجکت `QuantumCircuit` باشه.)

### ۱. استیت GHZ.

ورودی:
$$ n \in \mathbb{N} \space ; n \geq 1 $$

خروجی:

$$\frac{1}{\sqrt{2}} \big (|0\dots0\rangle + |1\dots1\rangle\big) = \frac{1}{\sqrt{2}} \big (|0\rangle^{\otimes n} + |1\rangle^{\otimes n}\big)$$

### ۲. سوپرپوزیشن بین bitstring و $|0\rangle^{\otimes n}$

ورودی: 

$$ s \in \{0, 1\}^{\otimes n} ; \space e.g \space (for \space n=2): 10 $$

خروجی:

$$ \frac{1}{\sqrt{2}}\big(|00\rangle + |10\rangle\big) $$

### ۳. سوپرپوزیشن بین دوتا bitstring

ورودی: 

$$ s_1, s_2 \in \{0, 1\}^{\otimes n} \space ; \space e.g \space (for \space n=2): \space 10, \space 01$$

خروجی:

$$ \frac{1}{\sqrt{2}}\big(|01\rangle + |10\rangle\big) $$

### ۴. این استیت:
$$\frac{1}{\sqrt{3}} \big(|00\rangle + |01\rangle + |10\rangle\big)$$

### ۵. استیت هاردی:
$$\frac{1}{\sqrt{12}} \big(3|00\rangle + |01\rangle + |10\rangle + |11\rangle\big)$$